# Vacancy Point Defects in Graphene

## 0. Introduction

This tutorial demonstrates the process of creating materials with vacancy defects, based on the work presented in the following manuscript:

> **Fabian Bertoldo, Sajid Ali, Simone Manti & Kristian S. Thygesen**, "
> "Quantum point defects in 2D materials - the QPOD database", Nature, 2022.  
> [DOI:10.1038/s41524-022-00730-w](https://doi.org/10.1038/s41524-022-00730-w)

Below is the figure 1 from the manuscript demonstrating the boron vacancy defects in hexagonal boron nitride (hBN).

In this notebook we will reproduce material from FIG. 6


<img src="https://i.imgur.com/VsBdZ7g.png" alt="vacancy in hBN" width="400"/>



## 1. Prepare the Environment
### 1.1. Set up defects parameters 
Defect Configuration parameters are described in [Defect Configuration](https://github.com/Exabyte-io/made/blob/8196b759242551c77d1791bf5bd2f4150763cfef/src/py/mat3ra/made/tools/build/defect/configuration.py#L102).

Note that we use approximate coordinates in crystal coordinates to define the defect positions. The approximate coordinates are used to find the closest site in the crystal to the given coordinates. The approximate coordinates are in the range of [0, 1] for each axis. Adjust the approximate coordinates to place the defect at the desired site in the crystal. Using the visuals provided in the notebook could help in determining the approximate coordinates.
(Coordinates found using 3D editor in Materials Designer)

In [ ]:
WIDTH = 8
LENGTH = 14
VACUUM_WIDTH = 0
VACUUM_LENGTH = 0

DEFECT_CONFIGS = [
    {
        "defect_type": "vacancy",
        "approximate_coordinate": [0.5, 0.5, 0.5],
        "use_cartesian_coordinates": False
    },

]

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("specific_examples|create_point_defect.ipynb")

### 1.3. Load input material
In this notebook we will use the material from the `uploads` folder that has a few pre-set materials.

In [ ]:
from mat3ra.made.material import Material
from mat3ra.standata.materials import Materials

material = Material(Materials.get_by_name_and_categories("Boron_Nitride", "2D"))

### 1.4. Create and preview Nanoribbon

In [ ]:
from utils.visualize import visualize_materials as visualize
from mat3ra.made.tools.build.nanoribbon import NanoribbonConfiguration, create_nanoribbon

config = NanoribbonConfiguration(
    material=material,
    width=WIDTH,
    length=LENGTH,
    vacuum_width=VACUUM_WIDTH,
    vacuum_length=VACUUM_LENGTH,
    edge_type="zigzag")

nanoribbon = create_nanoribbon(config)

visualize(nanoribbon, repetitions=[1, 1, 1], rotation="0x")
visualize(nanoribbon, repetitions=[1, 1, 1], rotation="-90x")

## 2. Create the Defect
### 2.1. Initialize Configuration and Builder parameters

In [ ]:
from mat3ra.made.tools.build.defect import PointDefectConfiguration
from mat3ra.made.tools.build.defect.builders import PointDefectBuilderParameters

defect_configurations = [PointDefectConfiguration.from_dict(nanoribbon, defect) for defect in DEFECT_CONFIGS]

defect_builder_parameters = PointDefectBuilderParameters(center_defect=False)

### 2.2. Create the defects

In [ ]:
from mat3ra.made.tools.build.defect import create_defects

material_with_defect = create_defects(
    builder_parameters=defect_builder_parameters,
    configurations=defect_configurations
¬)

## 3. Visualize Result(s)

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([{"material": nanoribbon, "title": "Original material"},
           {"material": material_with_defect, "title": f"Material with defect"}],
          rotation="-90x")
visualize([{"material": nanoribbon, "title": "Original material"},
           {"material": material_with_defect, "title": f"Material with defect"}])

## 4. Write resulting material to the folder

In [ ]:
from utils.jupyterlite import download_content_to_file

material_with_defect.name = "N-doped Graphene"
download_content_to_file(material_with_defect.to_json(), "B-vacancy_hexagonal_boron_nitride.json")